**UpVote if you find helpful**

In [1]:
!pip install -q autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 922.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 7.3 MB/s eta 0:00

In [2]:
# Import Library
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor


def add_feature_cross_terms(df, numerical_features):
    df_new = df.copy()
    for i in range(len(numerical_features)):
        for j in range(i + 1, len(numerical_features)):  
            feature1 = numerical_features[i]
            feature2 = numerical_features[j]
            cross_term_name = f"{feature1}_x_{feature2}"
            df_new[cross_term_name] = df_new[feature1] * df_new[feature2]
    return df_new




# Load data
train = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")



# Define numerical features
numerical_features = ["Age", "Height", "Weight", "Duration", "Heart_Rate", "Body_Temp"]



# Add cross terms to capture feature interactions
train = add_feature_cross_terms(train, numerical_features)
test = add_feature_cross_terms(test, numerical_features)




# Prepare training data with log-transformed target
train_data = train.copy()
train_data['log_Calories'] = np.log1p(train_data['Calories'])
train_data = train_data.drop(columns=['id', 'Calories'])




# Prepare test data, keeping 'id' for submission
test_data = test.copy()





# Initialize AutoGluon TabularPredictor
predictor = TabularPredictor(
    label='log_Calories',
    problem_type='regression',
    eval_metric='root_mean_squared_error',
    path='autogluon_model'
)



# Fit the model with bagging for robustness
predictor.fit(
    train_data,
    presets='best_quality',
    num_bag_folds=10,
    time_limit=17200  # Increase Time for better output
)

# Generate predictions on test data
log_preds = predictor.predict(test_data)

# Transform predictions back to original scale and clip
preds = np.expm1(log_preds)
preds = np.clip(preds, 1, 314)

# Submission file
submission['Calories'] = preds
submission.to_csv('submission.csv', index=False)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.80 GB / 31.35 GB (95.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

(_ray_fit pid=760) [1000]	valid_set's rmse: 0.0627012
(_ray_fit pid=761) [2000]	valid_set's rmse: 0.0589532 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=762) [2000]	valid_set's rmse: 0.057757 [repeated 2x across cluster]
(_ray_fit pid=761) [3000]	valid_set's rmse: 0.0587181 [repeated 2x across cluster]
(_ray_fit pid=759) [3000]	valid_set's rmse: 0.0592981
(_ray_fit pid=762) [3000]	valid_set's rmse: 0.0575848
(_ray_fit pid=761) [4000]	valid_set's rmse: 0.0586156 [repeated 2x across cluster]
(_ray_fit pid=762) [4000]	valid_set's rmse: 0.0575027 [repeated 2x across cluster]


(_ray_fit pid=760) 	Ran out of time, early stopping on iteration 4640. Best iteration is:
(_ray_fit pid=760) 	[4596]	valid_set's rmse: 0.0616363


(_ray_fit pid=1020) [1000]	valid_set's rmse: 0.0626315 [repeated 2x across cluster]
(_ray_fit pid=1051) [1000]	valid_set's rmse: 0.0603104
(_ray_fit pid=1084) [1000]	valid_set's rmse: 0.0619564
(_ray_fit pid=1020) [2000]	valid_set's rmse: 0.0620239 [repeated 2x across cluster]
(_ray_fit pid=1051) [2000]	valid_set's rmse: 0.0598285
(_ray_fit pid=1085) [2000]	valid_set's rmse: 0.0601858
(_ray_fit pid=1020) [3000]	valid_set's rmse: 0.061818 [repeated 2x across cluster]
(_ray_fit pid=1051) [3000]	valid_set's rmse: 0.0596861
(_ray_fit pid=1085) [3000]	valid_set's rmse: 0.0599617
(_ray_fit pid=1020) [4000]	valid_set's rmse: 0.0617565 [repeated 2x across cluster]
(_ray_fit pid=1051) [4000]	valid_set's rmse: 0.0596282
(_ray_fit pid=1085) [4000]	valid_set's rmse: 0.0598454


(_ray_fit pid=1020) 	Ran out of time, early stopping on iteration 4700. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=1020) 	[4284]	valid_set's rmse: 0.0617387 [repeated 4x across cluster]
(_ray_fit pid=1084) 	Ran out of time, early stopping on iteration 4563. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=1084) 	[4240]	valid_set's rmse: 0.0612341 [repeated 2x across cluster]


...

(_ray_fit pid=2059) 	Ran out of time, early stopping on iteration 2627. Best iteration is:
(_ray_fit pid=2059) 	[2624]	valid_set's rmse: 0.0625169


(_ray_fit pid=2311) [1000]	valid_set's rmse: 0.0599981 [repeated 2x across cluster]
(_ray_fit pid=2310) [1000]	valid_set's rmse: 0.0598752 [repeated 3x across cluster]
(_ray_fit pid=2311) [2000]	valid_set's rmse: 0.0596073
(_ray_fit pid=2280) [2000]	valid_set's rmse: 0.0599623
(_ray_fit pid=2310) [2000]	valid_set's rmse: 0.0594981 [repeated 2x across cluster]


(_ray_fit pid=2280) 	Ran out of time, early stopping on iteration 2588. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=2280) 	[2562]	valid_set's rmse: 0.0598405 [repeated 4x across cluster]


(_ray_fit pid=2499) [1000]	valid_set's rmse: 0.0617226
(_ray_fit pid=2531) [1000]	valid_set's rmse: 0.0614183
(_ray_fit pid=2499) [2000]	valid_set's rmse: 0.0614506
(_ray_fit pid=2531) [2000]	valid_set's rmse: 0.0609654
(_ray_fit pid=2499) [3000]	valid_set's rmse: 0.0613537
(_ray_fit pid=2531) [3000]	valid_set's rmse: 0.0607985
(_ray_fit pid=2499) [4000]	valid_set's rmse: 0.0613273
(_ray_fit pid=2531) [4000]	valid_set's rmse: 0.0607268


(_ray_fit pid=2499) 	Ran out of time, early stopping on iteration 4377. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=2499) 	[4334]	valid_set's rmse: 0.0613072 [repeated 4x across cluster]
(_dystack pid=283) 	-0.0606	 = Validation score   (-root_mean_squared_error)
(_dystack pid=283) 	1229.5s	 = Training   runtime
(_dystack pid=283) 	378.1s	 = Validation runtime
(_dystack pid=283) Fitting model: LightGBM_BAG_L2 ... Training model for up to 129.78s of the 129.68s of remaining time.
(_ray_fit pid=2531) 	Ran out of time, early stopping on iteration 4298. Best iteration is:
(_ray_fit pid=2531) 	[4112]	valid_set's rmse: 0.0607116
(_dystack pid=283) 	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=2.75%)
(_ray_fit pid=2756) 	Ran out of time, early stopping on iteration 118. Best iteration is:
(_ray_fit pid=2756) 	[118]	valid_set's rmse: 0.0612018
(_ray_fit pid=2900) 	Ran out of time, early stopping

**Thank You !**